In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np 

from pathlib import Path

In [ ]:
data = pd.read_csv(Path('data') / 'ice_cream.csv')
data.rename(columns={'DATE': 'date', 'IPN31152N': 'production'}, inplace=True)
data.date = pd.to_datetime(data.date)
data.set_index('date', inplace=True)
start_date = pd.to_datetime('2010-01-01')
data = data[start_date:]

plt.title('Ice Cream Production Year by Year')
plt.ylabel('production')
plt.xlabel('year')
for year in range(2010, 2021):
    plt.axvline(pd.to_datetime(str(year) + '-01-01'), 
                color='black',
                linestyle='--')
plt.plot(data.production)
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from pandas.plotting import autocorrelation_plot


# acf = plot_acf(data.production, lags=120) # 12 months * 10 years = 120 lags
autocorrelation_plot(data.production)

# Based on decaying ACF we're probably dealing with an autoregressive process 

In [ ]:
pacf = plot_pacf(data.production, method='ywm')

## Based on PACF we should start with an autoregressive model on lags 1, 2, 3, 10, 13

In [ ]:
from statsmodels.tsa.ar_model import AutoReg
from sklearn.model_selection import train_test_split

# X = data.index.values
# y = data.production.values

train, test = data[:60].values, data[60:].values
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=21, test_size=0.1)

In [ ]:
from sklearn.metrics import mean_squared_error

model = AutoReg(train, lags=[1, 2, 3, 10, 13])
model_fit = model.fit()
predictions = model_fit.predict(start=len(train), end=len(train)+len(test)-1, dynamic=False)
print('Coefficients: %s' % model_fit.params)

plt.plot(predictions, label='predictions')
plt.plot(test, label='true values')
plt.legend()
plt.show()
f'MSE: {mean_squared_error(predictions, test)}'

## now let's do the same thing assuming 12-months seasonality:

In [ ]:
model = AutoReg(train, lags=[1, 2, 3, 10, 13], seasonal=True, period=12)
model_fit = model.fit()
predictions = model_fit.predict(start=len(train), end=len(train)+len(test)-1, dynamic=False)
# print('Coefficients: %s' % len(model_fit.params))

plt.plot(predictions, label='predictions')
plt.plot(test, label='true values')
plt.legend()
plt.show()
f'MSE: {mean_squared_error(predictions, test)}'

In [ ]:
# bingo... looks better and the MSE is way lower

### Dickey Fuller Test

In [ ]:
from statsmodels.tsa.stattools import adfuller

# augmented dickey-fuller (for AR models more comlex than AR1)

adf_test_value, p_value, used_lags, nobs, _, _ = adfuller(data['production'].values)
print(f'''adf statistic: {adf_test_value} (lower than zero means no unit-roots)
p value: {p_value}''')

In [ ]:
# let's try to retrain the model with the number of lags produced by the augmented dickey-fuller
# test

model = AutoReg(train, lags=used_lags, seasonal=True, period=12)
model_fit = model.fit()
predictions = model_fit.predict(start=len(train), end=len(train)+len(test)-1, dynamic=False)
# print('Coefficients: %s' % len(model_fit.params))

plt.plot(predictions, label='predictions')
plt.plot(test, label='true values')
plt.legend()
plt.show()

print(f'''MSE: {mean_squared_error(predictions, test)}
using "number of lags" from the aug-dickey-fuller test helped!''')

In [ ]:
def generate_ar_process(lags: int, coefs: list[float], length: int) -> np.array:
    if len(coefs) != lags:
        raise ValueError(f'number of coefs ({len(coefs)}) must match lags: ({lags})')
        
    coefs = np.array(coefs)
    series = [np.random.normal() for _ in range(lags)] # generate first lags (normally distributed)
    
    for _ in range(length - lags):
        previous_values = series[-lags:][::-1] # take 'lags' values from in reversed order
        new_value = np.sum(np.array(previous_values) * coefs + np.random.normal())
        series.append(new_value)
    return np.array(series)

lags = 3
for coef in [0.1, 0.2, 0.33333, 0.334, 0.5]:
    coefs = [coef, coef, coef]
    generated_tseries = generate_ar_process(lags, coefs, 100)
    plt.title(f'''AR process with {lags} lags and coefs sum to: {sum(coefs)}
    adf statistic: {adfuller(generated_tseries)[0]}
    p value: {adfuller(generated_tseries)[1]}''')
    plt.plot(generated_tseries)
    plt.show()


### process remains stationary as long as sum of coefs is lower than 1
### when it's 1 and higher it gets a clean trend (trend-stationary?) 
### when it's significantly higher than 1 it become a smooth exponential 

# S&P500 

In [ ]:
import yfinance 

spy_data = yfinance.Ticker('SPY')
spy_data = spy_data.history(period='1d', start='2010-01-01', end='2020-01-01')
closing_prices = spy_data[['Close']]
plt.title('SPY daily closing price')
plt.plot(closing_prices)
plt.show()

In [ ]:
daily_diffs = closing_prices.values[1:] - closing_prices[:-1]
plt.title('SPY daily movements')
plt.ylabel('Price difference')
plt.plot(daily_diffs)
for year in range(2010, 2021):
    plt.axvline(pd.to_datetime(str(year) + '-01-01'), linestyle='--', color='gray')
plt.show()

In [ ]:
acf_plot = plot_acf(daily_diffs, lags=100)

In [ ]:
pacf_plot = plot_pacf(daily_diffs, method='ywm')

# no significant autocorrelation (predict stock-market movements is not easy)

In [ ]:
def get_ticker_data(ticker: str, start_date: str = '1920-01-01', end_date: str = '2022-01-01', period='1d'):
    ticker_data = yfinance.Ticker(ticker)
    ticker_data = ticker_data.history(period='1d', start=start_date, end=end_date)
    if len(ticker_data) == 0:
        raise Exception(f'no data for: {ticker}')
    return ticker_data[['Close']]

In [ ]:
market_data = pd.DataFrame()
start, end = '2013-11-06','2021-01-01'
market_data['SPY'] = get_ticker_data('SPY', start_date=start, end_date=end)
market_data['DAX'] = get_ticker_data('DAX', start_date=start, end_date=end)
sse_index_tracking_fund = 'ASHR' #Xtrackers Hvst CSI 300 China A-Shs ETF
market_data[sse_index_tracking_fund] = get_ticker_data(sse_index_tracking_fund, start_date=start, end_date=end)
market_data.plot() # market_data.plot() and not plt.plot(market_data) to get labels automatically from cols
plt.legend()
plt.show()

In [ ]:
market_data.corr() # correlation between markets seems between ~62% end ~66%

In [ ]:
import scipy.stats as stats

def clean_mutual_nans(df: pd.DataFrame, col1: str, col2: str):
    return df[df[col2].notnull()][col1].dropna(), df[df[col1].notnull()][col2].dropna()

ashr, dax = clean_mutual_nans(market_data, 'ASHR', 'DAX')
r, p_value = stats.pearsonr(ashr, dax)
print('pearson r:', r, 'p value: ', p_value) 

r, p_value = stats.pearsonr(market_data.interpolate(method='time').fillna(method='bfill')['ASHR'],
                            market_data.interpolate(method='time').fillna(method='bfill')['DAX'])
print('(interpolated data) pearson r:', r, 'p value: ', p_value) 

In [ ]:
# plot moving averages
tsla_data = pd.DataFrame()
tsla_data['price'] = get_ticker_data('TSLA', start_date='2010-01-01', end_date='2022-01-01')
tsla_data['100d-MA'] = tsla_data['price'].rolling(100).mean()
tsla_data['200d-MA'] = tsla_data['price'].rolling(200).mean()
tsla_data.dropna()[-100:].plot()
plt.show()

In [ ]:
volatility_data = get_ticker_data('VIX', start_date='2015-01-01', end_date='2018-01-01').apply(lambda x: x / 100)
volatility_data['SPY'] = get_ticker_data('SPY', start_date='2015-01-01', end_date='2018-01-01')
volatility_data.plot()
plt.show()

In [ ]:
def calc_roi(stock_price: pd.DataFrame):
    return stock_price.values[-1] / stock_price.values[0]

spy = get_ticker_data('SPY')
calc_roi(spy[pd.to_datetime('2020-12-01'):pd.to_datetime('2021-09-01')])

## search for TLCC: Time Lagged Cross Correlation — assessing signal dynamics

In [ ]:
spy, ashr = clean_mutual_nans(market_data, 'SPY', 'ASHR')
corr_df = pd.DataFrame()
for shift_periods in [10, 20, 30, 50]:
# shift_periods = 30
    corr_df['spy'] = spy
    corr_df['spy_shift'] = spy.shift(periods=shift_periods, freq='1D')
    corr_df['ashr'] = ashr
    corr_df['ashr_shift'] = ashr.shift(periods=shift_periods, freq='1D')
    assert corr_df.corr()['spy']['ashr_shift'] < corr_df.corr()['spy']['ashr']
    assert corr_df.corr()['ashr']['spy_shift'] < corr_df.corr()['ashr']['spy']
# doesn't look like there's some kind of TLCC...
corr_df.corr()

In [ ]:
assert corr_df.corr()['spy']['ashr_shift'] < corr_df.corr()['spy_shift']['ashr']
# also looks like more information is flowing from ashr to spy than the other way round 